In [38]:
# Experiment 2 - 1 output layer with 1 loss function - mse. and do hyper parameter tuning.
from utils.data_preprocess import load_data, load_single_leakage_model_data
from utils.module import model_eval, hyper_model, model_comparison, linear_regression, numpy_to_tensor, benchmark_linear_model
import itertools
import pandas as pd 
import yaml
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Dense, Input
import keras_tuner as kt
from tensorflow import keras
from keras import layers
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [39]:
model_path = 'saved_model/Multi_leak/1_loss_linear_reg/'
project_name='multileak_6out_1loss_withswap'

In [40]:
with open("config_multi.yml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)


single_leakage, two_leakage = load_data()
two_leakage["leak_1"] = 1
two_leakage["leak_2"] = 1

single_leakage["leak_1"] = 1
single_leakage["leak_2"] = 0

data = pd.concat([single_leakage, two_leakage], axis=0)
data['x2'] = data['x2'].replace(np.nan, 8024)
data['y2'] = data['y2'].replace(np.nan, 2616.5)

data = data.drop(columns=['mfc6_residual',
       'mfc7_residual', 'mfc8_residual', 'mfc9_residual', 'mfc10_residual',
       'mfc1_residual', 'mfc2_residual', 'mfc3_residual', 'mfc4_residual',
       'mfc5_residual', 'total flow rate'])

print(data.columns)
print(data.isna().sum())

y = data[['x1', 'y1', 'x2', 'y2', 'leak_1', 'leak_2']]
x = data.drop(['x1', 'y1', 'x2', 'y2', 'leak_1', 'leak_2'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1) 

y1_train = y_train[['x1', 'y1', 'x2', 'y2']]
y2_train = y_train[['leak_1', 'leak_2']]
y1_test = y_test[['x1', 'y1', 'x2', 'y2']]
y2_test = y_test[['leak_1', 'leak_2']]
y1_val = y_val[['x1', 'y1', 'x2', 'y2']]
y2_val = y_val[['leak_1', 'leak_2']]

def coords_swap(y1):
    s = y1['x2'] < y1['x1']
    y1.loc[s, ['x1','x2']] = y1.loc[s, ['x2','x1']].values
    y1.loc[s, ['y1','y2']] = y1.loc[s, ['y2','y1']].values
    return y1

y1_data = [y1_train, y1_val, y1_test]
y1_data_types = ['y1_train', 'y1_val', 'y1_test']
for y1_data_types, y1 in zip(y1_data_types, y1_data):
    y1_data_types = coords_swap(y1)

y1_columns = y1_train.columns
y2_columns = y2_train.columns
X_columns = X_train.columns

scaler_coords1 = StandardScaler()
y1_train = scaler_coords1.fit_transform(y1_train)
y1_test = scaler_coords1.transform(y1_test)
y1_val = scaler_coords1.transform(y1_val)

y1_train = pd.DataFrame(y1_train, columns=y1_columns)
y1_test = pd.DataFrame(y1_test, columns=y1_columns)
y1_val = pd.DataFrame(y1_val, columns=y1_columns)

# y1_train['x2'] = y1_train['x2'].replace(np.nan, -5)
# y1_train['y2'] = y1_train['y2'].replace(np.nan, -5)

# y1_test['x2'] = y1_test['x2'].replace(np.nan, -5)
# y1_test['y2'] = y1_test['y2'].replace(np.nan, -5)

# y1_val['x2'] = y1_val['x2'].replace(np.nan, -5)
# y1_val['y2'] = y1_val['y2'].replace(np.nan, -5)
# Not sure if 0 is good enough or try generating a random number

# scaler_coords2 = StandardScaler()
# y2_train = scaler_coords2.fit_transform(y2_train)
# y2_test = scaler_coords2.fit_transform(y2_test)
# y2_val = scaler_coords2.transform(y2_val)

# y2_train = pd.DataFrame(y2_train, columns=y2_columns)
# y2_test = pd.DataFrame(y2_test, columns=y2_columns)
# y2_val = pd.DataFrame(y2_val, columns=y2_columns)

y2_train = y2_train.reset_index().drop(columns='sample_number')
y2_val = y2_val.reset_index().drop(columns='sample_number')
y2_test = y2_test.reset_index().drop(columns='sample_number')

y_train_sc = pd.concat([y1_train, y2_train], axis=1)
y_test_sc = pd.concat([y1_test, y2_test], axis=1)
y_val_sc = pd.concat([y1_val, y2_val], axis=1)

scaler_flows = StandardScaler()
X_train = scaler_flows.fit_transform(X_train)
X_test = scaler_flows.transform(X_test)
X_val = scaler_flows.transform(X_val)

Index(['x1', 'y1', 'MFC6', 'MFC7', 'MFC8', 'MFC9', 'MFC10', 'MFC1', 'MFC2',
       'MFC3', 'MFC4', 'MFC5', 'leak_1', 'leak_2', 'x2', 'y2'],
      dtype='object')
x1        0
y1        0
MFC6      0
MFC7      0
MFC8      0
MFC9      0
MFC10     0
MFC1      0
MFC2      0
MFC3      0
MFC4      0
MFC5      0
leak_1    0
leak_2    0
x2        0
y2        0
dtype: int64


In [41]:
# Fitting a simple linear regression model
reg = LinearRegression().fit(X_train, y_train_sc)

In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
y_predictions_train = reg.predict(X_train)
# print("train", "{:10.4f}".format(mean_squared_error(y_train, y_predictions, squared=True)))
y_predictions_val = reg.predict(X_val)
# print("val", "{:10.4f}".format(mean_squared_error(y_val, y_predictions, squared=True)))
y_predictions = reg.predict(X_test)

y_predictions_train[:,-2][np.abs(y_predictions_train[:,-2]) < 0.5] = 0
y_predictions_train[:,-2][np.abs(y_predictions_train[:,-2]) > 0.5] = 1
y_predictions_train[:,-1][np.abs(y_predictions_train[:,-1]) < 0.5] = 0
y_predictions_train[:,-1][np.abs(y_predictions_train[:,-1]) > 0.5] = 1

y_predictions_val[:,-2][np.abs(y_predictions_val[:,-2]) < 0.5] = 0
y_predictions_val[:,-2][np.abs(y_predictions_val[:,-2]) > 0.5] = 1
y_predictions_val[:,-1][np.abs(y_predictions_val[:,-1]) < 0.5] = 0
y_predictions_val[:,-1][np.abs(y_predictions_val[:,-1]) > 0.5] = 1

y_predictions[:,-2][np.abs(y_predictions[:,-2]) < 0.5] = 0
y_predictions[:,-2][np.abs(y_predictions[:,-2]) > 0.5] = 1
y_predictions[:,-1][np.abs(y_predictions[:,-1]) < 0.5] = 0
y_predictions[:,-1][np.abs(y_predictions[:,-1]) > 0.5] = 1

loss_test = "{:10.4f}".format(mean_squared_error(y_test_sc, y_predictions, squared=True))
metric_test = "{:10.4f}".format(mean_absolute_error(y_test_sc, y_predictions))

loss_val = "{:10.4f}".format(mean_squared_error(y_val_sc, y_predictions_val, squared=True))
metric_val = "{:10.4f}".format(mean_absolute_error(y_val_sc, y_predictions_val))

loss_train = "{:10.4f}".format(mean_squared_error(y_train_sc, y_predictions_train, squared=True))
metric_train = "{:10.4f}".format(mean_absolute_error(y_train_sc, y_predictions_train))

print(metric_test, metric_val, metric_train)

    0.2344     0.2108     0.2260


In [43]:
# # converting the predictions to certainity of 0 and 1 (May not be a good idea)
# y_pred[:,-2][np.abs(y_pred[:,-2]) < 0.5] = 0
# y_pred[:,-2][np.abs(y_pred[:,-2]) > 0.5] = 1
# y_pred[:,-1][np.abs(y_pred[:,-1]) < 0.5] = 0
# y_pred[:,-1][np.abs(y_pred[:,-1]) > 0.5] = 1

# y_pred = pd.DataFrame(y_pred, columns=y_train.columns)
# y1_pred_inverse = scaler_coords1.inverse_transform(y_pred[['x1', 'y1','x2', 'y2']])
# y_pred[['x1', 'y1','x2', 'y2']] = pd.DataFrame(y1_pred_inverse,columns=['x1', 'y1','x2', 'y2'])
# mse = score_mse(y_test, y_pred)
# y_pred.loc[y_pred['leak_2'] == 0.0, 'x2'] =  'NaN'
# y_pred.loc[y_pred['leak_2'] == 0.0, 'y2'] =  'NaN'

# y_test.loc[y_test['leak_2'] == 0.0, 'x2'] =  'NaN'
# y_test.loc[y_test['leak_2'] == 0.0, 'y2'] =  'NaN'
# pd.concat([y_pred, y_test.reset_index().drop(columns='sample_number')], axis=1).to_csv(model_path+'predictions.csv')

In [44]:
#Define the model using model builder from keras tuner
def model_builder_single(hp):
    model = keras.Sequential()

    # Choose an optimal value between 32-512
    for i in range(hp.Int("num_layers", 1, 15)):
        l1_weight = hp.Choice('l1_weight', values=[0.0, 1e-1, 1e-2, 1e-3])
        l2_weight = hp.Choice('l2_weight', values=[0.0, 1e-1, 1e-2, 1e-3])
        kernel_regularizer=keras.regularizers.L1L2(l1 = l1_weight, l2 = l2_weight)
        model.add(
            keras.layers.Dense(
                units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32),
                activation=hp.Choice("activation", ["relu", "tanh"]),
                kernel_initializer='he_uniform',
                kernel_regularizer=kernel_regularizer
            )
        )
    model.add(keras.layers.Dense(units=6, activation= "linear", kernel_initializer='he_uniform'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    # hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-1, sampling="log")

    # model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    #             loss="mse",  metrics='mae')
    
    model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=learning_rate),
                loss="mse",
                metrics='mae')

    return model


#search for the best hyperparameters and train the standard model with original training data
def hyper_model(X_train,Y_train, X_val, y_val, epoch, factor):
    folder_name = project_name
    tuner = kt.Hyperband(model_builder_single,
                         objective='val_loss',
                         max_epochs=epoch+200,
                         factor=factor,
                         hyperband_iterations = 1,
                        # Integer, at least 1, the number of times to iterate over the full Hyperband algorithm. One iteration will 
                        # run approximately max_epochs * (math.log(max_epochs, factor) ** 2) cumulative epochs across all trials. It is 
                        # recommended to set this to as high a value as is within your resource budget. Defaults to 1.
                         directory="../../tensorflow_log_files/studienarbeit/",
                         seed=0,    
                         project_name=str(folder_name))

    tuner.search_space_summary()
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    tuner.search(X_train, Y_train, epochs=epoch, validation_data = (X_val, y_val), callbacks=[stop_early, 
                                                                                            #   keras.callbacks.TensorBoard("../tensorflow_log_files/studienarbeit/tb_logs"+str(folder_name))
                                                                                              ])
    #tuner.search(X_train, Y_train, epochs=50, validation_data=(X_test,Y_test), callbacks=[stop_early])
    # Get the optimal hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    model = tuner.hypermodel.build(best_hps)
    history = model.fit(X_train, Y_train, epochs=epoch, validation_data = (X_val, y_val), shuffle= True)

    print(f"""
    The hyperparameter search is complete. The optimal learning rate for the optimizer
    is {model.optimizer.lr.numpy()}.
    """)

    return best_hps, model, tuner, history

In [45]:
best_hps, best_model, tuner, history = hyper_model(X_train,y_train_sc, X_val, y_val_sc,
                                                        epoch=1000,factor=2)

INFO:tensorflow:Reloading Tuner from ../../tensorflow_log_files/studienarbeit/multileak_6out_1loss_withswap/tuner0.json


INFO:tensorflow:Reloading Tuner from ../../tensorflow_log_files/studienarbeit/multileak_6out_1loss_withswap/tuner0.json


Search space summary
Default search space size: 20
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 15, 'step': 1, 'sampling': 'linear'}
l1_weight (Choice)
{'default': 0.0, 'conditions': [], 'values': [0.0, 0.1, 0.01, 0.001], 'ordered': True}
l2_weight (Choice)
{'default': 0.0, 'conditions': [], 'values': [0.0, 0.1, 0.01, 0.001], 'ordered': True}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.1, 'step': None, 'sampling': 'log'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'con

INFO:tensorflow:Oracle triggered exit


Epoch 1/100
17/17 [==============================] - 3s 20ms/step - loss: 0.5086 - mae: 0.5344 - val_loss: 0.1895 - val_mae: 0.3336
Epoch 2/100
17/17 [==============================] - 0s 9ms/step - loss: 0.1298 - mae: 0.2637 - val_loss: 0.1217 - val_mae: 0.2556
Epoch 3/100
17/17 [==============================] - 0s 9ms/step - loss: 0.0765 - mae: 0.1979 - val_loss: 0.0891 - val_mae: 0.2124
Epoch 4/100
17/17 [==============================] - 0s 9ms/step - loss: 0.0571 - mae: 0.1671 - val_loss: 0.0807 - val_mae: 0.1920
Epoch 5/100
17/17 [==============================] - 0s 9ms/step - loss: 0.0452 - mae: 0.1457 - val_loss: 0.0722 - val_mae: 0.1855
Epoch 6/100
17/17 [==============================] - 0s 9ms/step - loss: 0.0383 - mae: 0.1344 - val_loss: 0.0701 - val_mae: 0.1833
Epoch 7/100
17/17 [==============================] - 0s 10ms/step - loss: 0.0333 - mae: 0.1236 - val_loss: 0.0669 - val_mae: 0.1718
Epoch 8/100
17/17 [==============================] - 0s 9ms/step - loss: 0.0298 -

In [46]:
y_pred = best_model.predict(X_test)
y_pred_train = best_model.predict(X_train)
y_pred_val = best_model.predict(X_val)

5/5 [==============================] - 0s 3ms/step


In [47]:
best_model.evaluate(X_train, y_train_sc)
best_model.evaluate(X_test, y_test_sc)
best_model.evaluate(X_val, y_val_sc)

5/5 [==============================] - 0s 3ms/step - loss: 0.0549 - mae: 0.1384


[0.054904562326678126, 0.13837788856672573]

In [48]:
# best_model.save(model_path)

In [49]:

# # converting the predictions to certainity of 0 and 1 (May not be a good idea)
# y_pred[:,-2][np.abs(y_pred[:,-2]) < 0.5] = 0
# y_pred[:,-2][np.abs(y_pred[:,-2]) > 0.5] = 1
# y_pred[:,-1][np.abs(y_pred[:,-1]) < 0.5] = 0
# y_pred[:,-1][np.abs(y_pred[:,-1]) > 0.5] = 1

# y_pred = pd.DataFrame(y_pred, columns=y_train.columns)
# y1_pred_inverse = scaler_coords1.inverse_transform(y_pred[['x1', 'y1','x2', 'y2']])
# y_pred[['x1', 'y1','x2', 'y2']] = pd.DataFrame(y1_pred_inverse,columns=['x1', 'y1','x2', 'y2'])
# mse = score_mse(y_test, y_pred)
# y_pred.loc[y_pred['leak_2'] == 0.0, 'x2'] =  'NaN'
# y_pred.loc[y_pred['leak_2'] == 0.0, 'y2'] =  'NaN'

# y_test.loc[y_test['leak_2'] == 0.0, 'x2'] =  'NaN'
# y_test.loc[y_test['leak_2'] == 0.0, 'y2'] =  'NaN'
# pd.concat([y_pred, y_test.reset_index().drop(columns='sample_number')], axis=1).to_csv(model_path+'predictions_nn.csv')